# README.md

###For each pt with an ADT ending date, find its T date and T value around 6months later

# Step1, read from spreadsheet, clean up, and get statistics

In [2]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
from datetime import timedelta, date
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

In [3]:
df1=pd.read_csv("Pt_with_ADT_window_940.csv", header=0)
df1.fillna('', inplace=True)
df1['inital_ADT_end_date'] = pd.to_datetime(df1['inital_ADT_end_date'])

df2=pd.read_excel("pca_tresults_20210119_BP.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)

In [10]:
df1[:1]

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days,index1
0,2,2009-02-26,2017-02-01,2897 days 00:00:00.000000000,2897,0


In [11]:
df2[:1]

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351


In [12]:
list_pts_adt_table=df1["person_id"].to_list()
list_pts_tvalue_table=df2["person_id"].to_list()

In [13]:
print("pt number in adt table is:")
print(len(list_pts_adt_table))
print("")
print("pt number in t value table is:")
print(len(list_pts_tvalue_table))

pt number in adt table is:
1514

pt number in t value table is:
18643


In [ ]:
index_list=df2.columns
index_list

In [ ]:
print(df2.shape)
#delete some useless columns
del_col=['cancer_type_id', 'year_of_diagnosis', 'month_of_diagnosis', 'day_of_diagnosis', 'dx_date','loinc_code', 'source_test_name']

for col in del_col:
    del df2[col]
print(df2.shape)

In [ ]:
y=2

df1_pt=df1[(df1["person_id"] == y)]
df2_pt=df2[(df2["person_id"] == y)]

In [ ]:
df1_pt

In [ ]:
df2_pt

In [ ]:
tvalue=df2_pt.loc[df2_pt['result_date'] == ('2017-02-01 00:00:00')]

In [ ]:
test=df2_pt[:1].test_results.values[0]
test

In [ ]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 

In [ ]:
y=2
df1_pt=df1[(df1["person_id"] == y)]
a=df1_pt['inital_ADT_end_date'].to_list()[0]
a

In [ ]:
a_plus_6mons=a + timedelta(days=180)

In [ ]:
a_plus_6mons

In [ ]:
df2_pt=df2[(df2["person_id"] == 2)]    
each_pt_tdates = df2_pt['result_date'].to_list()
#each_pt_tdates

In [ ]:
result_list=[]

#for y in list_pts:      
for y in [2]:
    
    df1_pt=df1[(df1["person_id"] == y)]
    a=df1_pt['inital_ADT_end_date'].to_list()[0]
    #add 6 months to the inital ADT end date
    a_plus_6mons=a + timedelta(days=180)
    #pick up initial ADT window 
    gap=df1_pt['Inital_ADT_time_window_days'].to_list()[0] 
    
    df2_pt=df2[(df2["person_id"] == y)]    
    each_pt_tdates = df2_pt['result_date'].to_list()

    ##get rid of nan
    cleaned_a=[]
    cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
    cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
    cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
    
    start_sorted_bin=sorted(cleaned_aaa)
    
    if(len(start_sorted_bin)==0):
        tdate='NaN'  

    if(len(start_sorted_bin)>0):
        #apply the closet function
        tdate=closest(start_sorted_bin,a_plus_6mons)
        #tvalue=df2_pt.loc[df['result_date'] == tdate].test_result
    
    #pick up T-value for T-date
    df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]
    tvalue=df2_pt_pick.test_results.values[0]
    
    print(y, a, gap, a_plus_6mons, tdate, tvalue)
    result_per_pt=[y, a, gap, a_plus_6mons, tdate, tvalue]
    result_list.append(result_per_pt)

In [ ]:
#do it for all pts in the list
#get t values as well
#only keep those within 5-7 months

In [ ]:
result_list=[]

#for y in list_pts:      
for y in list_pts_adt_table:
    
    df1_pt=df1[(df1["person_id"] == y)]
    a=df1_pt['inital_ADT_end_date'].to_list()[0]
    #add 6 months to the inital ADT end date
    a_plus_6mons=a + timedelta(days=180)
    #pick up initial ADT window 
    gap=df1_pt['Inital_ADT_time_window_days'].to_list()[0] 
    
    df2_pt=df2[(df2["person_id"] == y)]    
    each_pt_tdates = df2_pt['result_date'].to_list()

    ##get rid of nan
    cleaned_a=[]
    cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
    cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
    cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
    
    start_sorted_bin=sorted(cleaned_aaa)
    
    if(len(start_sorted_bin)==0):
        tdate='NaN'  

    if(len(start_sorted_bin)>0):
        #apply the closet function
        tdate=closest(start_sorted_bin,a_plus_6mons)
    
    #pick up Tvalue for Tdate
    df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]
    tvalue=df2_pt_pick.test_results.values[0]

    #print(y, a, gap, a_plus_6mons, tdate, tvalue)
    result_per_pt=[y, a, gap, a_plus_6mons, tdate,  tvalue]
    result_list.append(result_per_pt)

In [ ]:
result_list

In [ ]:
df_results=pd.DataFrame(result_list)
df_results.columns=(['person_id', 'inital_ADT_end_date', 'Inital_ADT_time_window_days', '6 mons after ADT', 'Tdate closest to 6 mons after ADT', "Tvalue"])

In [ ]:
#create a column for the time gap to screen the ones outside the 5mons to 7 mons window  
df_results['Tvalue_ADT+6mons_gap'] = df_results['Tdate closest to 6 mons after ADT'] - df_results['6 mons after ADT']
#create a column for the inital adt use window as days
df_results['Tvalue_ADT+6mons_gap_days']=df_results['Tvalue_ADT+6mons_gap'].dt.days

In [ ]:
df_results

In [ ]:
df_results.to_csv("Tdate_Tvalue_ADTwindow_closest_to_6mons_after_ADT_ends.csv", encoding='utf-8', index=False)

In [ ]:
df_results_select1=df_results[(df_results["Tvalue_ADT+6mons_gap_days"] <= 30)]
df_results_select2=df_results_select1[(df_results_select1["Tvalue_ADT+6mons_gap_days"] >= -30)]
print(len(df_results))
print(len(df_results_select1))
print(len(df_results_select2))

In [ ]:
df_results

In [ ]:
#pts cohort with T date and values within the window of 5 mons to 7 mons after inital ADT ends
df_results_select2

In [ ]:
#Fix the T values that contain <; plot the histograms and make the presentation slides

In [ ]:
###Need to fix this later

#df_results_select3=df_results_select2
#df_results_select3['Tvalue'][ind].str[1:]
#fixed_string_list=[]
#for ind in df_results_select2.index:
 
#    if (df_results_select3['Tvalue'][ind].str.contains('<')): 
#        fixed_string_list.append(ind)  
#        df_results_select3['Tvalue'][ind]=df_results_select3['Tvalue'][ind].str[1:]

#df_results_select3.loc[df_results_select3['Tvalue'].str.contains('<'), '<']=""
#s[1:]

In [ ]:
#plot the histograms and make the slides; 30 mins for each
#statistics
#logic #results #crosscheck 

In [ ]:
df4=pd.read_excel("Pca_Tdate_Tvalue_ADTwindow_clean_results_113_pts.xls", header=0)

In [ ]:
index_list=df4.columns
index_list

In [ ]:
#delete some columns
print(df4.shape)    
del_col=['6 mons after ADT','Tvalue_ADT+6mons_gap', 'Tvalue_ADT+6mons_gap_days']
for col in del_col:
    del df4[col]
print(df4.shape)

In [ ]:
df4[:5]

In [ ]:
gap = df4[["Inital_ADT_time_window_days"]].reset_index()
ax=gap.plot.scatter(x= "index", y="Inital_ADT_time_window_days", c='DarkBlue')
print(gap.max())
print(gap.min())

In [ ]:
###plot the ADT window histogram

sns.set_style("darkgrid")
sns.set_context("poster", font_scale=0.6, rc={"lines.linewidth": 2})
binsize=180

bins=np.arange(0, 4000, binsize)
fig, ax = plt.subplots(figsize = (8,5))

#n=df_results['Inital_ADT_time_window_days'].plot(kind = "hist", bins=bins, color="purple", alpha=0.5)
(n, bins, patches) = plt.hist(gap["Inital_ADT_time_window_days"], bins=bins,color="green", alpha=0.4)

#decorate

plt.title("Bin Size="+ str(int(binsize/30)) +" months")
plt.xlabel("Months", fontsize=16, fontweight="bold",color="gray")
plt.ylabel("Population", fontsize=16,fontweight="bold", color="gray")

ax.set_xticks(np.arange(0, 4000, 360))
ax.set_xticklabels(np.arange(0, 133, 12))


# plot the label/text to each bin
for i in range(0, len(n)):
    x_pos = (bins[i + 1] - bins[i])/4 + bins[i]
    y_pos = n[i] + 0.5
    label = str(int(n[i])) # relative frequency of each bin
    ax.text(x_pos, y_pos, label,fontsize=16,fontweight="bold",color="gray" )
    ax.xaxis.tick_bottom()


In [ ]:
Tvalue = df4[["Tvalue"]].reset_index()
ax=Tvalue.plot.scatter(x= "index", y="Tvalue", c='Magenta')
print(Tvalue.max())
print(Tvalue.min())

In [ ]:
###plot the T value histogram

sns.set_style("darkgrid")
sns.set_context("poster", font_scale=0.6, rc={"lines.linewidth": 2})
binsize=75

bins=np.arange(0, 1100, binsize)
fig, ax = plt.subplots(figsize = (8,5))

#n=df_results['Inital_ADT_time_window_days'].plot(kind = "hist", bins=bins, color="purple", alpha=0.5)
(n, bins, patches) = plt.hist(Tvalue["Tvalue"], bins=bins,color="green", alpha=0.4)

#decorate

plt.title("Bin Size="+ str(int(binsize)))
plt.xlabel("T-value", fontsize=16, fontweight="bold",color="gray")
plt.ylabel("Population", fontsize=16,fontweight="bold", color="gray")

ax.set_xticks(np.arange(0, 1100, 300))
ax.set_xticklabels(np.arange(0, 1100, 300))

# plot the label/text to each bin
for i in range(0, len(n)):
    x_pos = (bins[i + 1] - bins[i])/4 + bins[i]
    y_pos = n[i] + 0.5
    label = str(int(n[i])) # relative frequency of each bin
    ax.text(x_pos, y_pos, label,fontsize=16,fontweight="bold",color="gray" )
    ax.xaxis.tick_bottom()



In [ ]:
###plot the T value histogram

sns.set_style("darkgrid")
sns.set_context("poster", font_scale=0.6, rc={"lines.linewidth": 2})
binsize=70

bins=np.arange(0, 1100, binsize)
fig, ax = plt.subplots(figsize = (8,5))

#n=df_results['Inital_ADT_time_window_days'].plot(kind = "hist", bins=bins, color="purple", alpha=0.5)
(n, bins, patches) = plt.hist(Tvalue["Tvalue"], bins=bins,color="green", alpha=0.4)

#decorate

plt.title("Bin Size="+ str(int(binsize)))
plt.xlabel("T-value", fontsize=16, fontweight="bold",color="gray")
plt.ylabel("Population", fontsize=16,fontweight="bold", color="gray")

ax.set_xticks(np.arange(0, 1100, 280))
ax.set_xticklabels(np.arange(0, 1100, 280))


# plot the label/text to each bin
for i in range(0, len(n)):
    x_pos = (bins[i + 1] - bins[i])/4 + bins[i]
    y_pos = n[i] + 0.5
    label = str(int(n[i])) # relative frequency of each bin
    ax.text(x_pos, y_pos, label,fontsize=16,fontweight="bold",color="gray" )
    ax.xaxis.tick_bottom()

In [ ]:
Correlation = df4[["Inital_ADT_time_window_days","Tvalue"]].reset_index()
plt.matshow(Correlation.corr())

In [ ]:
ax=Correlation.plot.scatter(x= "Inital_ADT_time_window_days", y="Tvalue", c='green')
plt.axhline(y=75, color='r', linestyle='-')
